In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

In [2]:
df1 = pd.read_csv('{파일경로}/lifelog_data_2018.csv')
df2 = pd.read_csv('{파일경로}/lifelog_data_2019.csv')
df3 = pd.read_csv('{파일경로}/lifelog_data_2020.csv')

In [3]:
def engineering(n, d):
    # 성별 인코딩
    d['gender'] = d['gender'].apply(lambda x:0 if x == 'M' else 1)
    # 날짜 변환
    d['date'] = pd.to_datetime(d['date'])

    # 스트레스(target)
    d['pmStress'] = d['pmStress'] - 1
    # XGBoost의 pred 값이랑 맞추기 위해 target 값에 -1
    
    # 감정변화비율 = 오후감정/오전감정
    d['emotionChangeRate'] = d['pmEmotion'] / d['amEmotion']
    # 긍정변화평균 = emotion1~7의 평균
    d['positiveMean'] = d.filter(regex='Positive').mean(axis=1)
    # 긴장변화평균 = emotion1~7의 평균
    d['tensionnMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 긍정비율 = Positive5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['positiveRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 긴장비율 = Tension5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['tensionRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 긍정감정의 임시 테이블
    pos_temp = d.filter(regex='Positive')
    # 가장 높은 긍정감정의 숫자(1~7)
    d['topPositive'] = pos_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긍정감정의 숫자(1~7)
    d['botPositive'] = pos_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긍정 수치 1~7 * count 수
    for p, c in enumerate(pos_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (p+1)
        
    del pos_temp
    
    # 긴장감정의 임시 테이블
    ten_temp = d.filter(regex='Tension')
    # 가장 높은 긴장감정의 숫자(1~7)
    d['topTension'] = ten_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긴장감정의 숫자(1~7)
    d['botTension'] = ten_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긴장 수치 1~7 * count 수
    for t, c in enumerate(ten_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (t+1)
    
    del ten_temp
    
    # 수치 반영 긍정 평균
    d['positiveWMean'] = d.filter(regex='Positive').mean(axis=1)
    # 수치 반영 긴장 평균
    d['tensionWMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 수치 반영 긍정 비율
    d['positiveWRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 수치 반영 긴장 비율
    d['tensionWRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 최고 수치를 반영한 오전 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aCtPT'] = round(d['amEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오전 컨디션 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aEtPT'] = round(d['amCondition'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오후 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['pEtPT'] = round(d['pmEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    
    # (최고 수치를 반영한 오전 컨디션 + 최고 수치를 반영한 오전 감정 + 최고 수치를 반영한 오후 감정) / 3
    # (최고 수치를 반영한 오전 컨디션 + 최고 수치를 반영한 오전 감정 + 최고 수치를 반영한 오후 감정)
    d['aCEpEtPTm'] = round((d['aCtPT'] + d['aEtPT'] + d['pEtPT']) / 3)
    d['aCEpEtPTm3'] = (d['aCtPT'] + d['aEtPT'] + d['pEtPT'])
    
    # 긍정적인지 = 수치 반영 긍정 평균이 중앙값보다 크면 1 아니면 0
    d['positive'] = d['positiveWRate'].apply(lambda x:1 if x > d['positiveWRate'].median() else 0)
    # 부정적인지 = 수치 반영 긍정 평균이 중앙값보다 작으면 1 아니면 0
    d['negative'] = d['positiveWRate'].apply(lambda x:1 if x < d['positiveWRate'].median() else 0)
    
    # 긴장 상태인지 = 수치 반영 긴장 평균이 중앙값보다 크면 1 아니면 0
    d['aroused'] = d['tensionWRate'].apply(lambda x:1 if x > d['tensionWRate'].median() else 0)
    # 편안한 상태인지 = 수치 반영 긴장 평균이 중앙값보다 작으면 1 아니면 0
    d['relaxed'] = d['tensionWRate'].apply(lambda x:1 if x < d['tensionWRate'].median() else 0)
    
    # 활동 비율 = 자전거, 도보의 합 / 운송수단, 가만히 있기의 합
    d['activityRate'] = d[['on_bicycle', 'on_foot']].sum(axis=1) / d[['in_vehicle', 'still']].sum(axis=1)
    # 0으로 나누기된 것을 0으로 치환 > inf 값을 가짐
    d.replace([np.inf, -np.inf], 0, inplace=True)
    d.dropna(inplace=True)
    d.sample(n=d.shape[0], random_state=1234)
    
    return d

In [4]:
for n, d in enumerate([df1, df2, df3]):
    globals() [f'df{n+1}'] = engineering(n, d.copy()) # 위에서 생성한 engineering 함수 적용
df = pd.concat([df1, df2, df3], axis=0) # 데이터 결합 
df = df[[c for c in df.columns if c not in ['pmStress']] + ['pmStress']] # pmStress 마지막 열로 이동
df.drop(['date', 'userId'], axis = 1, inplace = True) # dtae, userId 삭제

In [5]:
# 사용한 파라미터 조합
n_estimators = [10, 50, 100]
max_depth = [None, 5, 10, 15, 20, 50, 100, 200, 500]
max_features = [4, 10, 15, 20]
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [2, 5, 10, 20]
criterion = ['entropy', 'gini']

In [16]:
# 하이퍼파라미터
random_state = 8888 # 랜덤 시드 설정
n_est = [100] # 결정 트리 개수 설정
max_d = [None] # 결정 트리의 최대 깊이
max_f = [30] # 최대 특성 개수
min_s_s = [4] # 최소 분리 샘플 수 
min_s_l = [4] # 최소 잎새 샘플 수
c = ['entropy'] # 결정 트리 분리 기준 
test_accuracy_list = [] # 각 반복에서 테스트 셋의 정확도를 저장할 리스트
results = [] # 모든 반복에서 테스트 셋 정화도를 저장할 리스트

smote = SMOTE() # 오버 샘플링 기법 중 SMOTE 사용

X, y = df.iloc[:, :-1], df.iloc[:, -1]
# X : pmStress 컬럼을 제외한 데이터 
# y : 'pmStress'

for i in range(20):
    train = pd.DataFrame() # train data를 저장할 리스트
    valid = pd.DataFrame() # validation data를 저장할 리스트
    test = pd.DataFrame() # test data를 저장할 리스트
    for j in range(5):
        train = pd.concat([train, df.loc[df['pmStress'] == np.unique(df['pmStress'])[j]][:round(len(df) / 5 * 0.9)]], axis=0) # 90% test data
        test = pd.concat([test, df.loc[df['pmStress'] == np.unique(df['pmStress'])[j]][-round(len(df) / 5 * 0.1):]], axis=0) # 10% test data

    train = train.sample(n=train.shape[0]) # , random_state=1234)
    test = test.sample(n=test.shape[0], random_state=1234) # , random_state=1234)
    
    X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
    X_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]
    
    # 교차 검증 정확도를 달성하는 하이퍼파라미터 조합 선택
    best_score = 0
    best_params = None
    for n_estimators in n_est:
        for max_depth in max_d:
            for max_features in max_f:
                for min_samples_split in min_s_s:
                    for min_samples_leaf in min_s_l:
                        for criterion in c:
                            score_sum = 0
                            # 데이터를 5개의 fold로 나누고, Shuffle = True로 지정하여 데이터를 랜덤으로 섞음
                            # k-fold 교차검증을 수행하기 위해 train set과 validation data로 분리
                            for train_index, val_index in KFold(n_splits=5, shuffle=True, random_state=random_state).split(train):
                                X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
                                y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
                                # validation data가 아닌, train data에만 smote 기법을 사용
                                X_train_kf_resampled, y_train_kf_resampled = smote.fit_resample(X_train_kf, y_train_kf)
                                
                                clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                                                             min_samples_split=min_samples_split, criterion=criterion, n_jobs=-1)
                                clf.fit(X_train_kf_resampled, y_train_kf_resampled)
                                score_sum += accuracy_score(y_val_kf, clf.predict(X_val_kf)) #5번 교차검증의 정확도의 합
                            score_avg = score_sum / 5 # 5번 교차검증 정확도의 평균
                            if score_avg > best_score: # best_score보다 score_sum이 클 경우 best_score를 score_avg로 업데이트하고 헤딩 파라미터 조합을 best_params에 저장
                                best_score = score_avg # best_score와 score_avg 값이 동일할 경우, False
                                # 각 하이퍼파라미터의 값을 출력
                                best_params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_features': max_features, 
                                               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'criterion': criterion}
                            # best_score 의 하이퍼파라미터 값 출력
                            print(f"n_estimators: {n_estimators}, max_depth: {max_depth}, max_features: {max_features}, "
                                  f"min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}, criterion: {criterion}",
                                  f" -> accuracy: {score_avg}")
    # 하이퍼 파라미터 값을 갖고 랜덤포레스트 모델 적합                        
    clf = RandomForestClassifier(**best_params)
    clf.fit(X_train_kf, y_train_kf)
    # 하이퍼 파라미터 값을 갖고, test data로 예측한 y_pred 값
    y_pred_rf = clf.predict(X_test)
    # 실제 test값과 예측값 사이의 정확도
    accuracy = accuracy_score(y_test, y_pred_rf)
    results.append(accuracy)

    print(f"Iteration: {i+1}") # 반복 횟수
    print(f"Best params: {best_params}") # 하이퍼파라미터 조합
    print(f"Accuracy on test set: {accuracy}") # accuracy
    print('=='*70)

    test_accuracy_list.append(accuracy) # 반복횟수에 따른 정확도를 저장

# result_rf = pd.DataFrame({'Test Accuracy': test_accuracy_list}) # 데이터프레임 형식으로 test_accuracy_list 저장                           

n_estimators: 100, max_depth: None, max_features: 30, min_samples_split: 4, min_samples_leaf: 4, criterion: entropy  -> accuracy: 0.5227848101265822
Iteration: 1
Best params: {'n_estimators': 100, 'max_depth': None, 'max_features': 30, 'min_samples_split': 4, 'min_samples_leaf': 4, 'criterion': 'entropy'}
Accuracy on test set: 0.6761904761904762
n_estimators: 100, max_depth: None, max_features: 30, min_samples_split: 4, min_samples_leaf: 4, criterion: entropy  -> accuracy: 0.520253164556962
Iteration: 2
Best params: {'n_estimators': 100, 'max_depth': None, 'max_features': 30, 'min_samples_split': 4, 'min_samples_leaf': 4, 'criterion': 'entropy'}
Accuracy on test set: 0.5619047619047619
n_estimators: 100, max_depth: None, max_features: 30, min_samples_split: 4, min_samples_leaf: 4, criterion: entropy  -> accuracy: 0.5063291139240507
Iteration: 3
Best params: {'n_estimators': 100, 'max_depth': None, 'max_features': 30, 'min_samples_split': 4, 'min_samples_leaf': 4, 'criterion': 'entropy'

In [9]:
y_pred_rf = pd.DataFrame(y_pred_rf)
y_pred_rf.columns = ['RandomForest']
y_pred_rf.to_csv('RandomForest.csv')

In [10]:
type(y_test)
y_test.to_csv('y_test.csv')